In [ ]:
!pip install python-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 13.6 MB/s 


In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, HTTPError
import pycrfsuite

In [ ]:
#sentences list for Bangali data
sentences = list()
with open("/content/mypos_rmtag.txt") as f:
    sentences = f.readlines()
#Remove whitespace characters like `\n` at the end of each line
sentences = [x.strip() for x in sentences]

In [ ]:
len(sentences)

43196

In [ ]:
sentences = [s for s in sentences if len(s) > 5] # remove very short "sentences"

In [ ]:
len(sentences)

43148

In [ ]:
print(sentences[200]),len(sentences),

ဪ ၊ အခန်း နံပါတ် ၃၀၅ မှာ တည်း လို့ ရ ပါ တယ် ။ အေးအေးဆေးဆေး ရှိ ပါ တယ် ။


(None, 43148)

In [ ]:
prepared_sentences = list()
#apply loop on sentences list to select one sentence
for sentence in sentences:
    #create a list lengths that have length of every word in sentence
    lengths = [len(w) for w in sentence.split(" ")]
    positions = []

    next_pos = 0
    #apply loop on 'lengths' list to select one word length
    for length in lengths:
        #set next position to next word
        next_pos = next_pos + length
        #append every word position to positions list
        positions.append(next_pos)
    #remove white spaces from 'sentence' and make one single string and store in 'concatenated' 
    concatenated = sentence.replace(" ", "")
    #take every character from 'concatenated' string variable and store in 'chars' list
    chars = [c for c in concatenated]
    #generate labels for every character if it is at starting position then 1 else 0 
    labels = [0 if not i in positions else 1 for i, c in enumerate(concatenated)]
    #add every character with there labels into list 'prepared_sentences'
    prepared_sentences.append(list(zip(chars, labels)))
    
    
print([d for d in prepared_sentences[200]])

[('ဪ', 0), ('၊', 1), ('အ', 1), ('ခ', 0), ('န', 0), ('်', 0), ('း', 0), ('န', 1), ('ံ', 0), ('ပ', 0), ('ါ', 0), ('တ', 0), ('်', 0), ('၃', 1), ('၀', 0), ('၅', 0), ('မ', 1), ('ှ', 0), ('ာ', 0), ('တ', 1), ('ည', 0), ('်', 0), ('း', 0), ('လ', 1), ('ိ', 0), ('ု', 0), ('့', 0), ('ရ', 1), ('ပ', 1), ('ါ', 0), ('တ', 1), ('ယ', 0), ('်', 0), ('။', 1), ('အ', 1), ('ေ', 0), ('း', 0), ('အ', 0), ('ေ', 0), ('း', 0), ('ဆ', 0), ('ေ', 0), ('း', 0), ('ဆ', 0), ('ေ', 0), ('း', 0), ('ရ', 1), ('ှ', 0), ('ိ', 0), ('ပ', 1), ('ါ', 0), ('တ', 1), ('ယ', 0), ('်', 0), ('။', 1)]


In [ ]:
#here sentence is prepared_sentence and i is length of prepared_sentence
def create_char_features(sentence, i):
    #set initial feature set char as first char in prepared_sentence
    features = [
        'bias',
        'char=' + sentence[i][0] 
    ]
    #if i >=1 then go to previous character else append 'BOS' in features list 
    if i >= 1:
        features.extend([
            'char-1=' + sentence[i-1][0],
            'char-1:0=' + sentence[i-1][0] + sentence[i][0],
        ])
    else:
        features.append("BOS")
        
    if i >= 2:
        features.extend([
            'char-2=' + sentence[i-2][0],
            'char-2:0=' + sentence[i-2][0] + sentence[i-1][0] + sentence[i][0],
            'char-2:-1=' + sentence[i-2][0] + sentence[i-1][0],
        ])
        
    if i >= 3:
        features.extend([
            'char-3:0=' + sentence[i-3][0] + sentence[i-2][0] + sentence[i-1][0] + sentence[i][0],
            'char-3:-1=' + sentence[i-3][0] + sentence[i-2][0] + sentence[i-1][0],
        ])
    #if i+1 < len(sentence) then go to next character and set it to next character and set char to next two characters else append 'EOS' to features list
    if i + 1 < len(sentence):
        features.extend([
            'char+1=' + sentence[i+1][0],
            'char:+1=' + sentence[i][0] + sentence[i+1][0],
        ])
    else:
        features.append("EOS")
    #if first if condition satisfy then go to second and third if condition and do the same work for next characters    
    if i + 2 < len(sentence):
        features.extend([
            'char+2=' + sentence[i+2][0],
            'char:+2=' + sentence[i][0] + sentence[i+1][0] + sentence[i+2][0],
            'char+1:+2=' + sentence[i+1][0] + sentence[i+2][0],
        ])
    
    if i + 3 < len(sentence):
        features.extend([
            'char:+3=' + sentence[i][0] + sentence[i+1][0] + sentence[i+2][0]+ sentence[i+3][0],
            'char+1:+3=' + sentence[i+1][0] + sentence[i+2][0] + sentence[i+3][0],
        ])
    return features



def create_sentence_features(prepared_sentence):
    return [create_char_features(prepared_sentence, i) for i in range(len(prepared_sentence))]

def create_sentence_labels(prepared_sentence):
    return [str(part[1]) for part in prepared_sentence]

In [ ]:
len(prepared_sentences)

43148

In [ ]:
#X list for training model with sentence features 
X = [create_sentence_features(ps) for ps in prepared_sentences[:-20000]]
#y list for training model with sentence labels(0 or 1) 
y = [create_sentence_labels(ps)   for ps in prepared_sentences[:-20000]]

#X_test list for testing model with sentence features 
X_test = [create_sentence_features(ps) for ps in prepared_sentences[-2173:]]
#y_test list for testing model with sentence labels(0 or 1) 
y_test = [create_sentence_labels(ps)   for ps in prepared_sentences[-2173:]]

In [ ]:
len(X),len(y),len(X_test),len(y_test)

(23148, 23148, 2173, 2173)

In [ ]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X, y):
    trainer.append(xseq, yseq)

In [ ]:
trainer.set_params({
    'c1': 1.0, 
    'c2': 1e-3,
    'max_iterations': 500,
    'feature.possible_transitions': True
})

In [ ]:
#training model 
trainer.train('mm-word-segmentation-500.crfsuite')

In [ ]:
#open trained model
tagger = pycrfsuite.Tagger()
tagger.open('mm-word-segmentation-500.crfsuite')

In [ ]:
#segment sentence or word by trained model
def segment_sentence(sentence):
    #remove white spaces from sentence
    sent = sentence.replace(" ", "")
    #tag sentence by trained model or create sentence features 
    prediction = tagger.tag(create_sentence_features(sent))
    #assign 'complete' to empty string 
    complete = ""
    #apply for loop on taged sentence
    for i, p in enumerate(prediction):
        #if label of character in sentence is 1 then brack that word from that place and add into complete
        if p == "1":
            complete += " " + sent[i]
        #if label of character in sentence is 0 then add that word as it is into complete
        else:
            complete += sent[i]
    return complete

In [ ]:
#for check our model
sentence = "မိုးရွာထားလို့အခန်းထဲမှာစိုထိုင်းထိုင်းဖြစ်နေတယ်"
prediction = tagger.tag(create_sentence_features(sentence))
complete = ""
for i, p in enumerate(prediction):
  if p == "1":
    complete += " " + sentence[i]
  else:
    complete += sentence[i]
complete,sentence

('မိုးရွာ ထား လို့ အခန်း ထဲ မှာ စိုထိုင်း ထိုင်း ဖြစ် နေ တယ်',
 'မိုးရွာထားလို့အခန်းထဲမှာစိုထိုင်းထိုင်းဖြစ်နေတယ်')

In [ ]:
#print Segmented sentences or Segmented words
print(segment_sentence("ဒီနေ့ကမ္ဘာ့စံချိန်သစ်တင်ခဲ့တယ်။"))
print(segment_sentence("အဲဒီပစ္စည်းကကုန်တိုက်မှာတွေ့တဲ့ဟာကိုပုံစံယူလုပ်ထားတာ။"))

ဒီ နေ့ ကမ္ဘာ့ စံချိန် သစ်တင် ခဲ့ တယ် ။
အဲဒီ ပစ္စည်း က ကုန်တိုက် မှာ တွေ့ တဲ့ ဟာ ကို ပုံစံ ယူ လုပ် ထား တာ ။


In [ ]:
#use this if you want to convert whole text file into segmented text file that store in 'output.txt' file
outF = open("/content/output.txt", "w")

with open('validation.txt','r') as f:
  for line in f:
    l1 = ""
    l2 = ""
    if len(line)<=5:
      l1=line
      #outF.write(line)
      outF.write(l1)
    else:
      seg = segment_sentence(line)
      l2 = seg
      outF.write(l2)

In [78]:
import pandas as pd
from bs4 import BeautifulSoup as soup
df = pd.read_fwf('/content/output.txt', header=None)
len(df)

1000

In [79]:
tp = 0
fp = 0
fn = 0
n_correct = 0
n_incorrect = 0

for s in prepared_sentences[-2176:]:
    prediction = tagger.tag(create_sentence_features(s))
    correct = create_sentence_labels(s)
    zipped = list(zip(prediction, correct))
    tp +=        len([_ for l, c in zipped if l == c and l == "1"])
    fp +=        len([_ for l, c in zipped if l == "1" and c == "0"])
    fn +=        len([_ for l, c in zipped if l == "0" and c == "1"])
    n_incorrect += len([_ for l, c in zipped if l != c])
    n_correct   += len([_ for l, c in zipped if l == c])

In [80]:
print("Precision:\t" + str(tp/(tp+fp)))
print("Recall:\t\t" + str(tp/(tp+fn)))
print("Accuracy:\t" + str(n_correct/(n_correct+n_incorrect)))

Precision:	0.9592365822714987
Recall:		0.9540267719991222
Accuracy:	0.9799878175680821


<ul>
<li>'c1': 1.0, 
<li>'c2': 1e-3,
<li>'max_iterations': 100,

<li>Precision:	0.959476619002926
<li>Recall:		0.9534233048057933
<li>Accuracy:	0.9799116773685947
</ul>

<ul>
<li>'c1': 1.0, 
<li>'c2': 1e-3,
<li>'max_iterations': 50,
<li>Precision:	0.9594102054340623
<li>Recall:		0.9530941408821593
<li>Accuracy:	0.9798228471358595
</ul>

- max iter 300
- Precision:	0.9593931034482759
- Recall:		0.9539719113451832
- Accuracy:	0.9800131976345778
